In [72]:
import pandas as pd
import numpy as np
import re
import os

In [73]:
folder_name_work = input('FolderName:')

FolderName: redo8-full


In [74]:
input = "../../outputs/mining/" + folder_name_work + "/phobius/nr100.clstr"
out_filename = "../../outputs/mining/" + folder_name_work + "/phobius/entries_to_delete.txt"

In [75]:
# Read the file content
with open(input, 'r') as file:
    lines = file.readlines()

# Initialize variables to store clusters
clusters = []
current_cluster = []

# Parse through the lines and segregate clusters
for line in lines:
    if line.startswith('>Cluster'):
        # Save the previous cluster if not empty
        if current_cluster:
            clusters.append(current_cluster)
        # Start a new cluster
        current_cluster = [line]
    else:
        current_cluster.append(line)

# Append the last cluster
if current_cluster:
    clusters.append(current_cluster)

# Filter out clusters with no matches
filtered_clusters = []
for cluster in clusters:
    # Check if the cluster contains more than just the first sequence
    if len(cluster) > 2 or (len(cluster) == 2 and not cluster[1].strip().endswith('*')):
        filtered_clusters.append(cluster)

# Flatten the filtered clusters into a single list
output_lines = [line for cluster in filtered_clusters for line in cluster]

In [76]:
output_lines

['>Cluster 12\n',
 '0\t1522aa, >sp|O60242|AGRB3_HUM... *\n',
 '1\t1063aa, >tr|A0A804HJR2|A0A80... at 100.00%\n',
 '>Cluster 17\n',
 '0\t1420aa, >tr|A8DY43|A8DY43_DR... *\n',
 '1\t710aa, >tr|A0JQ58|A0JQ58_DR... at 100.00%\n',
 '>Cluster 21\n',
 '0\t1360aa, >tr|Q7KTA0|Q7KTA0_DR... *\n',
 '1\t967aa, >tr|E1NZB1|E1NZB1_DR... at 100.00%\n',
 '>Cluster 28\n',
 '0\t1280aa, >tr|G1KHR0|G1KHR0_AN... *\n',
 '1\t1225aa, >tr|A0A803TQD1|A0A80... at 100.00%\n',
 '>Cluster 52\n',
 '0\t1248aa, >tr|A1Z7F5|A1Z7F5_DR... *\n',
 '1\t780aa, >tr|A1Z7F6|A1Z7F6_DR... at 100.00%\n',
 '>Cluster 79\n',
 '0\t1215aa, >sp|Q5T848|MGLYR_HUM... *\n',
 '1\t1136aa, >tr|A0A3B3IUC3|A0A3B... at 100.00%\n',
 '>Cluster 157\n',
 '0\t905aa, >tr|A8K5P7|A8K5P7_HU... at 100.00%\n',
 '1\t1141aa, >tr|K9J9K9|K9J9K9_HU... *\n',
 '>Cluster 238\n',
 '0\t1035aa, >tr|A0A0P5YKL2|A0A0P... *\n',
 '1\t403aa, >tr|A0A164UIV2|A0A16... at 100.00%\n',
 '>Cluster 242\n',
 '0\t951aa, >sp|Q9BXB1|LGR4_HUMA... at 100.00%\n',
 '1\t1032aa, >tr|Q59ER8|Q59ER

In [77]:
# Initialize variables to store clusters
# Initialize variables to store clusters
data = []
cluster_name = ""

# Parse through the lines and segregate clusters
for line in output_lines:
    if line.startswith('>Cluster'):
        # Update the current cluster name
        cluster_name = line.strip().replace('>', '')
    else:
        # Extract relevant information
        parts = line.split(',')
        if len(parts) == 2:
            sequence_info = parts[0].split()
            aa_length = sequence_info[1].strip('aa')
            identifier_info = parts[1].split('|')
            id_type = identifier_info[0].strip().split('>')[1]  # 'sp' or 'tr'
            identifier = identifier_info[1]  # Identifier after the pipe |
            # Check if it has 100% or *
            match_type = '100%' if '100.00%' in parts[1] else '*' if '*' in parts[1] else ''
            # Append the data to the list
            data.append([cluster_name, int(aa_length), id_type, identifier, match_type])

# Convert the data into a DataFrame
df = pd.DataFrame(data, columns=['Cluster', 'length_aa', 'ID_Type', 'Identifier', 'Match_Type'])

In [78]:
# import taxonomy 
filename = "../../outputs/mining/" + folder_name_work + "/tree_metadata/complfulltree_map_all.txt"
prefiltered_tree = pd.read_csv(filename, sep=",")
taxa = prefiltered_tree[['Entry','Org']]

In [79]:
### merge with taxonomy 
mat = pd.merge(df, taxa,  left_on='Identifier', right_on= 'Entry', how='left')

In [80]:
#mat[mat['Cluster'] == 'Cluster 1966']

In [81]:
#mat[mat['Org'] == 'MICPS']

In [82]:
#mat['Cluster'].value_counts()

In [83]:
# Function to filter each group
def filter_group(group):
    ## it's only one taxonomic group
    if len(group['Org'].value_counts()) == 1: 
        # Filter to only rows with length_aa >= 560 (Technically GPCRs)
        # not very good approach
        #filtered_group = group[group['length_aa'] >= 560]
        
        # If there's more than one row, check for ID_Type and remove the longest one if ID_Type is 'tr'

        # get the swissprot
        # Filter the DataFrame to find rows where ID_type is 'sp'
        if (group['ID_Type'] == 'sp').any():
            filtered_df = group.loc[group['ID_Type'] == 'sp']
        # Further filter those rows where Match_Type is '*'
            # result_df = filtered_df.loc[filtered_df['Match_Type'] == '*']
        
        # If you want to return the first matching row
        #if not result_df.empty:
            return filtered_df
        else: 
            #print('This', group['Cluster'].iloc[0], 'has no Swiss-Prot equivalent. TrEMBL selection proceed.')
            
            #all are TrEMBL
            #check the length aa
            
            #if all has equal length, pick the *
            if group['length_aa'].nunique() == 1:
                # Further filter those rows where Match_Type is '*'
                result_df = group.loc[group['Match_Type'] == '*']
                return result_df
           #if the length is different, print these cluster has different length, need manual check!
            else: 
                group = group.drop_duplicates(subset='length_aa')
                if len(group['Entry'].value_counts()) == 2:
                    max_length = group['length_aa'].max()
                    filtered_group = group[group['length_aa'] == max_length]
                    return filtered_group
                
                else:
                    # Get the most frequent value in the column
                    most_frequent_value = group['length_aa'].value_counts().idxmax()
                    
                    filtered_rows = group[group['length_aa'] == most_frequent_value]
                    any_row = filtered_rows.iloc[0]
                    return any_row
            #if not tr_rows.empty: # if trembl entries for that cluster are not empty 
                # Remove the longest tr row
            #    max_length = tr_rows['length_aa'].max()
            #    filtered_group = filtered_group[~((filtered_group['length_aa'] == max_length) & (filtered_group['ID_Type'] == 'tr'))]
    
    else:
        if (group['Org'] == 'MICPS').any:
            max_length = group['length_aa'].max()
            filtered_group = group[group['length_aa'] == max_length]
            filtered_group = filtered_group.drop_duplicates(subset='length_aa')
            return filtered_group
        else:
            return print('This', group['Cluster'].iloc[0], 'has more than one taxonomic group. Need manual check!')

# Apply the filter to each group and concatenate the results
filtered_df = mat.groupby('Cluster').apply(filter_group).reset_index(drop=True)
filtered_df = filtered_df [['Cluster','length_aa','ID_Type','Identifier','Match_Type','Entry','Org']]

/tmp/ipykernel_22418/506290986.py:62: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = mat.groupby('Cluster').apply(filter_group).reset_index(drop=True)


In [84]:
filtered_df = filtered_df.dropna(how='any')

In [85]:
#original entries - after filtering
original_entries = filtered_df['Identifier'].tolist()

In [159]:
# Extract unique values from 'Cluster' column in both DataFrames
clusters_mat = set(mat['Cluster'].dropna())
clusters_filtered_df = set(filtered_df['Cluster'].dropna())

# Find missing values in filtered_df
missing_in_filtered_df = clusters_mat - clusters_filtered_df

print("Clusters in mat but missing in filtered_df. Need manual inspection:")
print(missing_in_filtered_df)
print(len(missing_in_filtered_df))

Clusters in mat but missing in filtered_df. Need manual inspection:
{'Cluster 403', 'Cluster 281', 'Cluster 2601', 'Cluster 284', 'Cluster 8255', 'Cluster 8603', 'Cluster 1966', 'Cluster 3894', 'Cluster 393', 'Cluster 412', 'Cluster 1682', 'Cluster 1200'}
12


In [143]:
# Filter the original DataFrame (mat) to include only rows with these missing clusters
extracted_rows = mat[mat['Cluster'].isin(missing_in_filtered_df)]
extracted_rows

,Cluster,length_aa,ID_Type,Identifier,Match_Type,Entry,Org
20,Cluster 281,400,tr,A0A0K3AS42,100%,A0A0K3AS42,CAEEL
21,Cluster 281,598,tr,A0A0K3AV56,100%,A0A0K3AV56,CAEEL
22,Cluster 281,1008,tr,A1IMB8,*,A1IMB8,CAEEL
23,Cluster 284,398,tr,A0A0K3AUI1,100%,A0A0K3AUI1,CAEEL
24,Cluster 284,596,tr,A0A0K3AXG3,100%,A0A0K3AXG3,CAEEL
25,Cluster 284,1006,tr,Q8MXU2,*,Q8MXU2,CAEEL
51,Cluster 393,536,tr,A0A0P5LWL2,100%,A0A0P5LWL2,9CRUS
52,Cluster 393,955,tr,A0A0P6FFT8,*,A0A0P6FFT8,9CRUS
53,Cluster 393,746,tr,A0A0P6GPG7,100%,A0A0P6GPG7,9CRUS
56,Cluster 403,952,tr,A0A0K3ASV1,*,A0A0K3ASV1,CAEEL


In [163]:
#input the selected entries and append them to the final entries file
# remaining_clusters = input('Selected entries and append to the final entries file:')
remaining_entries = 'A0A0K3AV56,A0A0K3AXG3,A0A0P6GPG7,A0A0K3AWH7,A0A0P5DB28,A0A0D2WJ40,A0A7S4L6U0,A0A7I4ETR6,Q86YG9,G5ED22,A0A7S3A2X8'
remaining_entries_list = remaining_entries.split(',')

In [172]:
## why i have two extra information?
len(original_entries)

646

In [167]:
# concatenate original_entries with remaining_entries
## just do it once
original_entries.extend(remaining_entries_list)

In [178]:
redundant_entries = mat[~mat['Identifier'].isin(original_entries)]

In [179]:
#out_filename: entries_to_delete
redundant_entries['Identifier'].to_csv(out_filename, header=False, index=False)

In [180]:
redundant_entries

,Cluster,length_aa,ID_Type,Identifier,Match_Type,Entry,Org
1,Cluster 12,1063,tr,A0A804HJR2,100%,A0A804HJR2,HUMAN
3,Cluster 17,710,tr,A0JQ58,100%,A0JQ58,DROME
5,Cluster 21,967,tr,E1NZB1,100%,E1NZB1,DROME
7,Cluster 28,1225,tr,A0A803TQD1,100%,A0A803TQD1,ANOCA
9,Cluster 52,780,tr,A1Z7F6,100%,A1Z7F6,DROME
...,...,...,...,...,...,...,...
1396,Cluster 13193,205,tr,E9PIN9,100%,E9PIN9,HUMAN
1398,Cluster 13206,266,tr,B2GQ52,100%,B2GQ52,DANRE
1400,Cluster 13302,255,tr,A0A178UXI2,100%,A0A178UXI2,ARATH
1402,Cluster 13386,229,tr,A0A4W3GYA2,100%,A0A4W3GYA2,CALMI


### Delete redundant entries

In [181]:
combined_entries = "../../outputs/mining/" + folder_name_work + "/phobius/phobius_filtered_list.txt"
entries_to_delete = "../../outputs/mining/" + folder_name_work + "/phobius/entries_to_delete.txt"
#output
filtered_entries = "../../outputs/mining/" + folder_name_work + "/phobius/filtered_entries.txt"

In [182]:
# Read the contents of the files
with open(combined_entries, 'r') as file_a:
    list_a = file_a.readlines()

with open(entries_to_delete, 'r') as file_b:
    list_b = file_b.readlines()

# Remove newline characters for accurate comparison
list_a = [line.strip() for line in list_a]
list_b = [line.strip() for line in list_b]

# Filter out the entries in list_a that are present in list_b
filtered_list = [item for item in list_a if item not in list_b]

# Optionally, write the filtered list back to file_a or a new file
with open(filtered_entries, 'w') as output_file:
    for item in filtered_list:
        output_file.write(item + '\n')

print("Filtered content has been saved to 'filtered_entries.txt'")

Filtered content has been saved to 'filtered_entries.txt'


# Updating or filtering (e.g. CREST superfamily, GPCR or other Uniprot) to generate a new list

## Further filtering the tree metadata information

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Set the working directory
folder_name_work = 'redo8-full'

In [3]:
# loading the tree file
#update to tree_metadata relabelled - 03.09.2024
filename = "../../outputs/mining/" + folder_name_work + "/tree_metadata/complfulltree_map_all.txt"
entries_to_delete = "../../outputs/mining/" + folder_name_work + "/phobius/entries_to_delete.txt"
not_7tm = "../../outputs/mining/" + folder_name_work + "/phobius/not_7tm.txt"

In [145]:
#this is the full
prefiltered_tree = pd.read_csv(filename, sep=",")
prefiltered_tree = prefiltered_tree.drop_duplicates(subset=['Entry'])
#prefiltered_tree.to_csv(filename,header=True, index=False)

In [146]:
len(prefiltered_tree)

17698

In [113]:
# Open and read the list of sequences that are not 7TM
with open(not_7tm, 'r') as file_c:
    list_c = [line.strip() for line in file_c]

# Remove sequences listed in 'not_7tm' from 'prefiltered_tree' - 'merge' with taxonomy
only_7tm = prefiltered_tree[~prefiltered_tree['Entry'].isin(list_c)]

In [114]:
#calling the redundant entries obtained with CD-HIT
with open(entries_to_delete, 'r') as file_b:
    list_b = [line.strip() for line in file_b]

### remove the 'entries to delete' - cd-hit step
non_redundant_tree = only_7tm[~only_7tm['Entry'].isin(list_b)]

In [115]:
## re-establish the complete and the filtered metadata
ln_add = pd.read_csv('../../outputs/mining/redo9-trash/tree_metadata/complfulltree_map_all.txt', sep = ',')
#remove duplicates in entry
tmp_ln = ln_add.drop_duplicates(subset =['Entry'])
just_ln = tmp_ln[['Entry','Length']]

In [116]:
#non_redundant_tree_lnx[non_redundant_tree_lnx['Entry'] == 'S4RWC1']
non_redundant_tree_lnx = non_redundant_tree.drop_duplicates(subset=['Entry'])

In [117]:
non_redundant_tree_ln = pd.merge(non_redundant_tree_lnx, just_ln,  left_on='Entry', right_on= 'Entry', how='left')

In [118]:
out_filename_filter_original = "../../outputs/mining/" + folder_name_work + "/tree_metadata/filtered_phobius_cdhit_RL.txt"
non_redundant_tree_ln.to_csv(out_filename_filter_original, header=True, index=False)

In [119]:
len(non_redundant_tree_ln)

13448

### Get a protein representative from each 'identical named protein'

In [120]:
## pick-one representative protein - taken from main Jupyter Notebook
# Sort by 'Protein names', 'Org', and 'Length' in descending order
gpcr_subset = non_redundant_tree_ln.sort_values(by=['Rename','Protein names', 'Scientific_Name', 'Length'], ascending=[True, True, True, False])

# Group by 'Protein names' and calculate the average length
gpcr_avg_length = gpcr_subset.groupby(['Rename','Scientific_Name','Protein names'], as_index=False)['Length'].mean()

In [121]:
## rename column name 
gpcr_avg_length = gpcr_avg_length.rename(columns={'Length': 'Average Length'})

In [122]:
# merge the average length
gpcr_subset = gpcr_subset.merge(gpcr_avg_length, on=['Rename', 'Scientific_Name', 'Protein names'], how = 'inner')

In [123]:
# How far is from the average - absolute value
gpcr_subset['Length Distance'] = (gpcr_subset['Average Length'] - gpcr_subset['Length']).abs()

# sort ascending
gpcr_subset = gpcr_subset.sort_values(by=['Rename', 'Scientific_Name', 'Protein names','Length Distance','Length'], ascending=[True,True,True,True,True])

In [124]:
# Check individual protein names
#pf01_subset_filtered[pf01_subset_filtered['Protein names'] == 'G-protein coupled receptor 55-like protein']

# Drop duplicates, keeping the first occurrence (the closest to the average, and has the lower length)
gpcr_subset = gpcr_subset.drop_duplicates(subset=['Rename','Scientific_Name','Protein names'])

In [125]:
out_filename_filter_original_subset = "../../outputs/mining/" + folder_name_work + "/tree_metadata/filtered_phobius_cdhit_RPP_RL.txt"
out_filename_filter_original_subset_et = "../../outputs/mining/" + folder_name_work + "/phobius/filtered_phobius_cdhit_RPP_entries.txt"

gpcr_subset.to_csv(out_filename_filter_original_subset, header=True, index=False)
gpcr_subset['Entry'].to_csv(out_filename_filter_original_subset_et, header=False, index=False)

In [126]:
len(gpcr_subset)

4643

## Remove a set of GPCR

In [127]:
# List of gpcr values to be deleted
delete_list = [
    'PF02117', 'PF02118', 'PF02175', 'PF03125', 'PF03383',
    'PF10292', 'PF10316', 'PF10317', 'PF10318', 'PF10319',
    'PF10320', 'PF10321', 'PF10322', 'PF10323', 'PF10324',
    'PF10325', 'PF10326', 'PF10327', 'PF10328'
]

# Filter the DataFrame to keep only the rows where 'gpcr' is not in the delete_list
not_serpentine = gpcr_subset[~gpcr_subset['gpcr'].isin(delete_list)]

In [128]:
len(not_serpentine)

4415

In [23]:
## export to csv - not serpentine 
#out_filename_filter1 = "../../outputs/mining/" + folder_name_work + "/tree_metadata/filtered1_not_serpentine_tree.txt"
#not_serpentine.to_csv(out_filename_filter1, header=True, index=False)

### export the entries only - not serpentine
#out_filename_filter2 = "../../outputs/mining/" + folder_name_work + "/phobius/filtered1_not_serpentine_tree.txt"
#not_serpentine['Entry'].to_csv(out_filename_filter2, header=False, index=False)

In [129]:
protein_ids_to_delete = [
    "C5KEK9", "Q12361", "Q12361", "A0A0L0FNK0", "Q9FHX6", "A0A0L0D6R5", 
    "A0A0L0DQV9", "A0A0M0JEL5", "A0A164WFC9", "A0A1L8EZ28", "A0A1L8G4K5", 
    "A0A1L8H190", "A0A2K3DRI3", "A0A3Q2UDN9", "A0A4W3HF08", "A0A4W3J0S4", 
    "A0A4W3KF93", "A0A7M7NQU0", "A0A803TQK9", "A7RQG1", "A7STF9", "B3S0I7", 
    "E1ZDN9", "G4VCE0", "H3AE54", "H3AL37", "H3ASW2", "H3AV51", "H3BB62", 
    "H9GC10", "H9GPR3", "Q07959", "K2NMZ6", "M3XKS7", "Q801G2", "Q7ZVH1", 
    "Q9NXK6", "Q86WK9", "Q8TEZ7", "Q6ZVX9", "Q4QJ02", "Q86D34", "Q94276", 
    "Q9VAS3", "R7T8U0", "S4R4X2", "S4RAW4", "A0A2P6NTJ8", "Q99393", "Q54J77", 
    "A0A1J4K029", "A0A6B2LAR8", "A0A6B2LBQ6", "A0A7S2MMB5", "A0A7S3ECK3", 
    "A0A7S3T1R1", "T2MBY2", "A0A7S2TXZ3", "A0A7S4AZD7", "A0A0H5RAX3", 
    "A0A1I9VZE3", "A0A1I9VZE4", "A0A1I9VZE8", "A0A2V2VBB6", "A0A7S0F1E6", 
    "A0A7S0IYQ4", "A0A7S2X5M9", "A0A7S3YDU6", "A0A7S4EWT9", "A0A7S4L6A8", 
    "A0A7S4MDD7", "A1L182", "B3KXW6", "B7ZQQ1", "C3Z226", "C3Z4J6", "K9J969", 
    "V9KKR4", "V9KZI5", "Q4V7P2", "Q9H345", "A0A7S4BT51", "Q9M9K7", 
    "A0A0A0RRC0", "A0A0I9QQP3", "A0A0N8BPY8", "A0A0P4XW15", "A0A0P5GGR3", 
    "A0A0P5MND9", "A0A0P6G2X6", "A0A126LAZ2", "A0A2D1AB92", "A0A2D1AQB8", 
    "A0A2H4P6E7", "A0A4Y1JWR6", "A0A5K4EWB4", "A0A678WD55", "A0A7D5D3Y5", 
    "A0A7T0BQX3", "A0A7T0BR20", "A0A7T0BRA0", "A0A7T0FY02", "A0PF30", 
    "A0PF31", "A0PF34", "A1IHG6", "A1KXD5", "A4IJ48", "A8K1T0", "A8K2J1", 
    "A8K2X9", "A8K4F0", "A8K5Q6", "A8K7T1", "A8K7X3", "A8K7Z2", "B2R9I6", 
    "B2R9U4", "B2RA17", "B2RE67", "B3KV13", "B3KX68", "B4DLS3", "C3YBJ5", 
    "C3YKN3", "C3ZJE5", "C6SUV0", "C8CG40", "D1LX50", "D1LX51", "D1LX82", 
    "D1LX84", "D1LXE5", "D2CFN9", "E1BVI5", "F2W7D9", "G4XH60", "H5V8A1", 
    "H9NIL3", "K4FYC0", "K9LKS7", "M9NJI3", "O76873", "O77269", "Q09K97", 
    "Q09K98", "Q09K99", "Q09KA0", "Q1RKV7", "Q24511", "Q3KNS9", "Q3SAH2", 
    "Q4VBN6", "Q5U791", "Q5XG06", "Q6JA19", "Q6NWR5", "Q6NWS6", "Q6XCE1", 
    "Q6ZMP9", "Q76L88", "Q8I943", "Q9BYT4", "Q9U5A7", "T2FFR0", "T2MDX2", 
    "T2MGT5", "T2MJN7", "V9KGD4", "V9KH48", "V9L3P5", "V9L539", "V9LBZ6", 
    "A0A7S1TA08", "A0A7S2TGW8", "A8KA23", "V9KKI8", "V9KZ28", "Q9UVN2", 
    "Q9UVN3", "A0A086JH98", "A0A086LPA6", "A0A7S4B8B4", "A0A7S4PFB1", 
    "A0A0N8BFX6", "A0A5K1UMW4", "A0A0I9QQ00", "F1QTF7", "Q86SE3", "A0A7S4I236", 
"Q54J71","Q6DC77","A0A833SXE6","A0A3Q0KR83","P55085"]

In [130]:
# Filter out rows where 'id_column' is in the list of IDs
filter_expassy1 = not_serpentine[~not_serpentine['Entry'].isin(protein_ids_to_delete)]

In [131]:
len(filter_expassy1)

4234

In [65]:
### expassy 1
out_filename_filter3 = "../../outputs/mining/" + folder_name_work + "/phobius/filtered1_expassy.txt"
filter_expassy1['Entry'].to_csv(out_filename_filter3, header=False, index=False)

## Renaming the tree_metadata

In [132]:
# Define your list of IDs
id_list_git3 = ["L8HLR0_ACACF", "A0A2P6N124_9EUKA", "F1A2E5_DICPU", "A0A1X7UQJ0_AMPQE",
    "H3B6G6_LATCH", "GP157_HUMAN", "A0A4W3GN33_CALMI", "R7ULM5_CAPTE",
    "A0A1X7URB2_AMPQE", 
               "A0DKL9_PARTE", "W7X9L3_TETTS", "A0D9W5_PARTE","L8H869_ACACF", ## goood classification
               "L8HF38_ACACF", "A0A0L0DIQ1_THETB", "A0A0L0SXM6_ALLM3",
               "A0A0L0DY10_THETB", "CRLF_DICDI",
               "A7RV25_NEMVE"] #new
# Replace values in 'gpcr' and 'color' columns
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_git3), 'Rename'] = 'Git3'
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_git3), 'gpcr'] = 'PF11710'

# Define your list of IDs
id_list_camp = [
    "CRLC_DICDI", "A7S541_NEMVE", "A0A1X7VI69_AMPQE", 
    "CRLG_DICDI", "F1A0N4_DICPU", 
"A0A5J4YHZ4_PORPP", "M2X2Z8_GALSU","A0A162PLB8_9CRUS","L8GKI9_ACACF"]

# Replace values in 'gpcr' and 'color' columns
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_camp), 'Rename'] = 'cAMP'
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_camp), 'gpcr'] = 'PF05462'

# Define your list of IDs
id_list_friz = [
    "A0A0L0SZS7_ALLM3", "A0A0L0SPS2_ALLM3"]

# Replace values in 'gpcr' and 'color' columns
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_friz), 'Rename'] = 'Friz'
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_friz), 'gpcr'] = 'PF01534'

# Define your list of IDs
id_list_camp_pipk = [
    "A0A0D2WMD5_CAPO3", "A0A833WNI1_PHYIN","A9UUR7_MONBE",
"A0A833WGZ5_PHYIN","A0A0L0DRJ2_THETB", "A0A2P6NGK7_9EUKA",
"A0A2P6NIK2_9EUKA"]

# Replace values in 'gpcr' and 'color' columns
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_camp_pipk), 'Rename'] = 'cAMP'
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_camp_pipk), 'gpcr'] = 'PF05462'

# Define your list of IDs
id_list_git3_pipk = [
    "A0A833RTJ3_PHYIN", "L8GPK2_ACACF","Q86D86_DICDI",
"F0ZZ14_DICPU"]

# Replace values in 'gpcr' and 'color' columns
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_git3_pipk), 'Rename'] = 'Git3'
filter_expassy1.loc[filter_expassy1['Entry Name'].isin(id_list_git3), 'gpcr'] = 'PF11710'

In [137]:
len(filter_expassy1)

4234

In [133]:
## filter tree topology 2
## Remove a set of GPCR
# List of gpcr values to be deleted
#fungi_list = [
#    'PF02076', 'PF02116'
#]

# Filter the DataFrame to keep only the rows where 'gpcr' is not in the delete_list
#not_fungi = filter_expassy1[~filter_expassy1['gpcr'].isin(fungi_list)]

In [134]:
#out_filename_filter4 = "../../outputs/mining/" + folder_name_work + "/phobius/filtered_not_fungi.txt"
#not_fungi['Entry'].to_csv(out_filename_filter4, header=False, index=False)

In [135]:
#not_fungi = filter_expassy1 ### but including the pheromone receptors

## taking out CAEEL
#not_caeel = not_fungi[not_fungi['Org'] != 'CAEEL']

In [136]:
#out_filename_filter5 = "../../outputs/mining/" + folder_name_work + "/phobius/filtered_test.txt"
#not_caeel['Entry'].to_csv(out_filename_filter5, header=False, index=False)

In [138]:
### second round expassy deletion
#old deletion
#protein_ids_to_delete_2 = [
#    "E2IJA8", "Q6MGK6", "A0A0D2X067", "X1W3V0", "Q23WQ7", "A0D441",
#    "L8GRC2", "L8H6T9", "A0A833T7I8", "A0A5K1U9Q6", "A0A2P6NHR2",
#    "A0A6U0LRW9", "A0A7J6K537", "A0A086L423", "Q9VBG4", "A0A5K1V1E8",
#    "R7V6P8", "A0A2P6NME1"
#]

#new deletion
protein_ids_to_delete_2 = [
    'G5EDK5', 'Q9VBG4', 'P34352', 'A0A5K4F4J1', 'A0A0L1I9U5', 'A0A7S4KMU9',
    'Q24265', 'P22815', 'Q8MSJ2', 'A0A164P9U8', 'A0A5K1V1E8', 
    'R7UBB3', 'Q10907', 'A0A3Q0KN47', 'Q6MGK6', 'F0Z8D0', 
    'Q23WQ7', 'X1W3V0', 'J3QSD0', 'A0A1L8FBK3', 'A0A1L8FBD9', 
    'A0A833T7I8', 'L8H6T9', 'L8GRC2'
]


In [139]:
#this has caeel
not_caeel = filter_expassy1 ### but including the pheromone receptors

# Filter out rows where 'id_column' is in the list of IDs
filter_expassy2 = not_caeel[~not_caeel['Entry'].isin(protein_ids_to_delete_2)]

In [141]:
len(filter_expassy2)

4210

In [21]:
#filter_expassy2[filter_expassy2['gpcr'] == 'PF02076']

In [22]:
#out_filename_filter6 = "../../outputs/mining/" + folder_name_work + "/phobius/full_mod_filtered2_expassy.txt"
#out_filename_filter6 = "../../outputs/mining/" + folder_name_work + "/phobius/full_mod_filtered2_expassy_with_caeel.txt"
#filter_expassy2['Entry'].to_csv(out_filename_filter6, header=False, index=False)

In [142]:
#filter_expassy2.to_csv("../../outputs/mining/" + folder_name_work + "/tree_metadata/full_filtered2_expassy.txt", header=True, index=False)
filter_expassy2.to_csv("../../outputs/mining/" + folder_name_work + "/tree_metadata/subset_expassy2.txt", header=True, index=False)

In [148]:
### second round expassy deletion - from the redo5-additional-with caeel 

protein_ids_to_delete_3 = [
    "A0A6B2L9L7", "A0A162R3I2","Q54WN6","Q54K09","Q54K11","Q54F54","S4S1R4","A0A0R4IP11","H3BIQ7"
]

In [150]:
filter_expassy3 = filter_expassy2[~filter_expassy2['Entry'].isin(protein_ids_to_delete_3)]

out_filename_filter7 = "../../outputs/mining/" + folder_name_work + "/tree_metadata/subset_expassy3.txt"
filter_expassy3.to_csv(out_filename_filter7, header=True, index=False)

'redo5-additional'

In [11]:
#this does not work 
only_characterised = not_serpentine[~not_serpentine['Protein names'].str.contains("Uncharacterized", case=False)]

In [14]:
out_filename_filter3 = "../../outputs/mining/" + folder_name_work + "/phobius/filtered1_only_characterised.txt"
only_characterised['Entry'].to_csv(out_filename_filter3, header=False, index=False)

In [24]:
#See if an species is available 
non_redundant_tree[non_redundant_tree['gpcr'] == 'PF11970']

,Entry Name,Entry,Organism,Org,Gene,Protein names,gpcr,taxa,Accession,Rename
4922,A0D441_PARTE,A0D441,Paramecium tetraurelia,PARTE,A0D441,"Chromosome undetermined scaffold_37, whole gen...",PF11970,2759,PF11970,Git3-2
4924,GPR1_YEAST,Q12361,Saccharomyces cerevisiae (strain ATCC 204508 /...,YEAST,GPR1,G protein-coupled receptor GPR1,PF11970,2759,PF11970,Git3-2
4925,A0A1X7V4F0_AMPQE,A0A1X7V4F0,Amphimedon queenslandica (Sponge),AMPQE,A0A1X7V4F0,G-protein coupled receptors family 2 profile 2...,PF11970,2759,PF11970,Git3-2
4926,L8HLI1_ACACF,L8HLI1,Acanthamoeba castellanii (strain ATCC 30010 / ...,ACACF,L8HLI1,G-protein coupled receptors family 2 profile 2...,PF11970,2759,PF11970,Git3-2
4930,A0A7S4AZD7_CHRCT,A0A7S4AZD7,Chrysotila carterae (Marine alga) (Syracosphae...,CHRCT,A0A7S4AZD7,G-protein coupled receptors family 2 profile 2...,PF11970,2759,PF11970,Git3-2


In [113]:
gpcr_only['Entry'].to_csv(out_filename_filter1, header=False, index=False)

### Keep just the GPCR

In [67]:
#list of crest gpcrs
gpcr_demendoza = ["PF00001", #rhod
                  "PF00002", #sec
                  "PF00003", #glu
                  "PF01534", #friz
                  "PF05462", #cAMP
                  "PF02101", #OA
                  "PF12430", #ABA
                  "PF10192", #ITR
                  "PF06814", #GOST
                  # "PF10324", #Srw
                  "PF03619", #Ostalpha
                 # "PF06454", #THH1,
                  "PF11710", #Git3
                  "PF11970" ] #Git3-2

In [68]:
### remove the 'crest family'
demendoza_only = non_redundant_tree[non_redundant_tree['gpcr'].isin(gpcr_demendoza)]

In [69]:
demendoza_only[demendoza_only['Entry'] == 'L8HLI1']

,Entry Name,Entry,Organism,Org,Gene,Protein names,gpcr,taxa,Accession,Rename
3149,L8HLI1_ACACF,L8HLI1,Acanthamoeba castellanii (strain ATCC 30010 / ...,ACACF,L8HLI1,G-protein coupled receptors family 2 profile 2...,PF11710,2759,PF11710,Git3


In [19]:
### filter according to the selected taxa
taxa_pisani = ["HUMAN","DANRE","CHICK","ANOCA","XENLA","LATCH","CALMI","PETMA","BRAFL","CION","SACKO","DROME","9CRUS","LIMPO","CAPTE",
               "SCHMA","HELRO","STRPU","LOTGI","NEMVE","ACRDI","HYDVU","ANTEL","ACRMI","APLCA","TRIAD","MNELE","OSCCA","AMPQE","CAPO3",
               "9EUKA","ALLMA","ALLM3","DICDI","DICPU","ACACA","ACACF","ARATH","MICPS","ECTSI","PARTE","TETTH","TETTS","BIGNA","PHATC",
               "DIALT","CHRCT","PORPP","CHOCR","CYAM1","GALSU","NEOHI","PORUM","9RHOD","9FLOR"]

In [20]:
taxa_pisani_only = filter_expassy2[filter_expassy2['Org'].isin(taxa_pisani)]
taxa_pisani_only

,Entry Name,Entry,Organism,Org,Scientific_Name,Gene,Protein names,gpcr,taxa,Accession,Rename
0,GPR22_DANRE,A0A2R9YJI3,Danio rerio (Zebrafish) (Brachydanio rerio),DANRE,Danio rerio,GPR22,G-protein coupled receptor 22,PF00001,2759,PF00001,Rho
1,APJB_DANRE,A0T2N3,Danio rerio (Zebrafish) (Brachydanio rerio),DANRE,Danio rerio,APJB,Apelin receptor B (Angiotensin II receptor-lik...,PF00001,2759,PF00001,Rho
2,A8JQT4_DROME,A8JQT4,Drosophila melanogaster (Fruit fly),DROME,Drosophila melanogaster,A8JQT4,"5-hydroxytryptamine (Serotonin) receptor 2A, i...",PF00001,2759,PF00001,Rho
3,A8JUP8_DROME,A8JUP8,Drosophila melanogaster (Fruit fly),DROME,Drosophila melanogaster,A8JUP8,Cholecystokinin-like receptor at 17D1,PF00001,2759,PF00001,Rho
4,MC4R_DANRE,B0V1P1,Danio rerio (Zebrafish) (Brachydanio rerio),DANRE,Danio rerio,MC4R,Melanocortin receptor 4 (MC4-R),PF00001,2759,PF00001,Rho
...,...,...,...,...,...,...,...,...,...,...,...
17591,A0A4W3GY61_CALMI,A0A4W3GY61,Callorhinchus milii (Ghost shark),CALMI,Callorhinchus milii,A0A4W3GY61,Transmembrane protein 187,PF15100,2759,PF15100,TMEM187
17593,A7RFT5_NEMVE,A7RFT5,Nematostella vectensis (Starlet sea anemone),NEMVE,Nematostella vectensis,A7RFT5,Transmembrane protein 187,PF15100,2759,PF15100,TMEM187
17594,C3YUE3_BRAFL,C3YUE3,Branchiostoma floridae (Florida lancelet) (Amp...,BRAFL,Branchiostoma floridae,C3YUE3,Transmembrane protein 187,PF15100,2759,PF15100,TMEM187
17596,G1KMB8_ANOCA,G1KMB8,Anolis carolinensis (Green anole) (American ch...,ANOCA,Anolis carolinensis,G1KMB8,Transmembrane protein 187,PF15100,2759,PF15100,TMEM187


In [22]:
taxa_pisani_only[taxa_pisani_only['gpcr'] == 'PF10327']

,Entry Name,Entry,Organism,Org,Scientific_Name,Gene,Protein names,gpcr,taxa,Accession,Rename


In [23]:
out_filename_taxa = "../../outputs/mining/" + folder_name_work + "/phobius/taxa_pisani_only_with_Git3.txt"
taxa_pisani_only['Entry'].to_csv(out_filename_taxa, header=False, index=False)